<a href="https://colab.research.google.com/github/Lou1108/DeepLearning/blob/main/Assignment2/Simplified_layers_meg_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


What we need to do
*   do the pre processing seperately and save the data
*   check and try different models to see if there's one working better with the data
* grid search on the number of the neurons in the layers



### Assignment 2

# Imports and Variables


In [ ]:
import os
import glob
import h5py
import numpy as np
from collections import Counter

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.signal import butter, filtfilt, decimate

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Input,

    Conv1D,
    Conv2D,

    DepthwiseConv2D,
    SeparableConv2D,

    MaxPooling1D,

    GlobalAveragePooling1D,
    AveragePooling2D,

    LSTM,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    Activation,
)


In [ ]:
TRAIN_PATH = "/content/drive/MyDrive/Deep Learning/Final Project data/Intra/train"
TEST_PATH = "/content/drive/MyDrive/Deep Learning/Final Project data/Intra/test"

# data specific
NUM_CHANNELS = 248
NUM_CLASSES = 4
LABEL_MAP = {'rest':0, 'task_motor':1, 'task_story_math':2, 'task_working_memory':3}
NUM_CLASSES = len(LABEL_MAP)
orig_fs=2034
target_fs=250
DOWNSAMPLE_FACTOR =  int(orig_fs / target_fs)

# Model specific
NUM_EPOCHS = 1
NORMALIZATION_METHOD = "time"  # Choose: "minmax", "zscore", or "perchannel"

In [ ]:
def load_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        # Extractin the label
        filename = file_path.split('/')[-1]

        #handling the different task naming conventions
        if 'rest' in filename:
            labels.append(LABEL_MAP['rest'])
        elif 'motor' in filename:
            labels.append(LABEL_MAP['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(LABEL_MAP['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(LABEL_MAP['task_working_memory'])
        else:
            # iff a file doesn't match
            print(f"Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            # Instead of guessing the dataset name, we get the first key from the file
            # This is robust because we know there is only one dataset per file[cite: 10].
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    #convert to numpy arrays
    return np.array(data), np.array(labels)

In [ ]:
train_files = glob.glob(f"{TRAIN_PATH}/*.h5")
test_files = glob.glob(f"{TEST_PATH}/*.h5")

X_train, y_train = load_data(train_files)
X_test, y_test = load_data(test_files)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Uniqe labels: {np.unique(y_train)}")
print(f"Number of training samples: {len(X_train)}")

Shape of X_train: (32, 248, 35624)
Shape of y_train: (32,)
Shape of X_test: (8, 248, 35624)
Uniqe labels: [0 1 2 3]
Number of training samples: 32


# Load and Preprocess Data
Apply a lowpass filter for downsampling the frequency

In [ ]:
# lowpass filter ---> check it
# def bandpass_filter(data, lowcut=1.0, highcut=150.0, fs=2034, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = butter(order, [low, high], btype='band')
#     return filtfilt(b, a, data, axis=-1)

Normalization functions

In [ ]:
def min_max_scale_sample(data):
    scaler = MinMaxScaler()
    return scaler.fit_transform(data.reshape(-1, 1)).reshape(data.shape)

def z_score_normalize(data):
    mean = data.mean(axis=-1, keepdims=True)
    std = data.std(axis=-1, keepdims=True)
    return (data - mean) / (std + 1e-8)

def time_norm(data):
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

In [ ]:
def normalization(data):
    if NORMALIZATION_METHOD == "minmax":
        data = min_max_scale_sample(data)
    elif NORMALIZATION_METHOD == "zscore":
        data = z_score_normalize(data)
    elif NORMALIZATION_METHOD == "time":
        data = time_norm(data)
    return data

In [ ]:
def normalize_data(data):
    # Data shape is (n_samples, n_channels, n_timesteps)
    # We want to scale each of the (n_samples * n_channels) time series

    # Reshape to (n_samples * n_channels, n_timesteps) to apply StandardScaler
    n_samples, n_channels, n_timesteps = data.shape
    reshaped_data = data.reshape(n_samples * n_channels, n_timesteps)

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(reshaped_data)

    # Reshape back to the original shape
    return scaled_data.reshape(n_samples, n_channels, n_timesteps)

In [ ]:
X_train.shape

(32, 248, 35624)

In [ ]:
N_TIMESTEPS = X_train.shape[2]
X_train_ds=decimate(X_train, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
X_test_ds=decimate(X_test, DOWNSAMPLE_FACTOR, axis=-1, ftype='fir', zero_phase=True)
X_train_ds = X_train[:, :, ::DOWNSAMPLE_FACTOR]
X_test_ds = X_test[:, :, ::DOWNSAMPLE_FACTOR]

N_TIMESTEPS_DS = X_train_ds.shape[2]

print(f"Original number of time steps: {N_TIMESTEPS}")
print(f"Downsampled number of time steps: {N_TIMESTEPS_DS}")

Original number of time steps: 35624
Downsampled number of time steps: 4453


In [ ]:
X_train_norm = normalization(X_train_ds)
X_test_norm = normalization(X_test_ds)

In [ ]:
#DL models in Keras often expect the channel dimension last
#reshaping from (samples, channels, timesteps) to (samples, timesteps, channels)
X_train_final = np.transpose(X_train_norm, (0, 2, 1))
X_test_final = np.transpose(X_test_norm, (0, 2, 1))

print(f"Final shape of training data for the model: {X_train_final.shape}")

Final shape of training data for the model: (32, 4453, 248)


In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True)
]

# Define Models

###CNN model

In [ ]:
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(16, 3, padding='same', activation='elu',
               kernel_regularizer=regularizers.l2(1e-2)),
        BatchNormalization(),
        GlobalAveragePooling1D(),
        Dropout(0.6),
        Dense(num_classes, activation='softmax')
    ])


    return model

In [ ]:
INPUT_SHAPE = (N_TIMESTEPS_DS, NUM_CHANNELS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 4453, 16)       │        11,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 4453, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,052 (47.08 KB)

 Trainable params: 12,020 (46.95 KB)

 Non-trainable params: 32 (128.00 B)

Training and evaluation of CNN

In [ ]:
history_cnn = cnn_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,      # smaller batch size for better generalization
    validation_split=0.2, # 20% of training data for validation
    callbacks=callbacks
)

# Evaluate on test Sets
loss_cnn, acc_cnn = cnn_model.evaluate(X_test_final, y_test, verbose=0)
print(f"accuracy on test set: {acc_cnn * 100:.2f}%")

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 865ms/step - accuracy: 0.2225 - loss: 2.6156 - val_accuracy: 0.2857 - val_loss: 1.3728
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 429ms/step - accuracy: 0.5133 - loss: 1.5443 - val_accuracy: 0.5714 - val_loss: 1.2053
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 236ms/step - accuracy: 0.8250 - loss: 0.7276 - val_accuracy: 0.5714 - val_loss: 1.0864
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step - accuracy: 0.6408 - loss: 0.8463 - val_accuracy: 0.5714 - val_loss: 0.9778
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.6142 - loss: 1.0983 - val_accuracy: 0.8571 - val_loss: 0.8782
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step - accuracy: 0.4925 - loss: 1.1008 - val_accuracy: 1.0000 - val_loss: 0.7975
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - accuracy: 0.8517 - loss: 0.7624 - val_accuracy: 1.0000 - val_loss: 0.7272
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 233ms/step - accuracy: 0.8517 - loss: 0.7614 - val_accuracy: 1.0000 - val_loss:

Training and evaluation LSTM Model

In [ ]:
def build_cnn_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),

        # 1 conv block
        Conv1D(filters=32, kernel_size=5, padding='same', activation='elu', kernel_regularizer=regularizers.l2(1e-3)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),

        # Lstm
        LSTM(units=32, return_sequences=False, kernel_regularizer=regularizers.l2(1e-3)),

        # Dense head
        Dense(32,activation='elu',kernel_regularizer=regularizers.l2(1e-3)),
        Dropout(0.5),
        # Softmax
        Dense(num_classes, activation='softmax')
    ])

    return model


In [ ]:
INPUT_SHAPE = (N_TIMESTEPS_DS, NUM_CHANNELS)

cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 4453, 32)       │        39,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4453, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 2226, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,348 (192.77 KB)

 Trainable params: 49,284 (192.52 KB)

 Non-trainable params: 64 (256.00 B)

In [ ]:
#CNN-LSTM Training and Evaluation
print("Starting CNN-LSTM model training...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2,
    callbacks=callbacks
)

# Evaluate on test sets
loss_hybrid, acc_hybrid = cnn_lstm_model.evaluate(X_test_final, y_test, verbose=0)
print(f"Hybrid Model Accuracy on Test Set: {acc_hybrid * 100:.2f}%")

Starting CNN-LSTM model training...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.3975 - loss: 1.3681 - val_accuracy: 0.5714 - val_loss: 1.1374
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.7833 - loss: 0.9175 - val_accuracy: 0.5714 - val_loss: 0.9444
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.9525 - loss: 0.7096 - val_accuracy: 1.0000 - val_loss: 0.8109
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 1.0000 - loss: 0.5548 - val_accuracy: 1.0000 - val_loss: 0.7248
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9258 - loss: 0.5958 - val_accuracy: 1.0000 - val_loss: 0.6655
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 1.0000 - loss: 0.4935 - val_accuracy: 1.0000 - val_loss: 0.6103
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.9467 - loss: 0.5851 - val_accuracy: 1.0000 - val_loss: 0.5549
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 1.0000 - loss: 0.3829 - val_accuracy: 1.0000

Training EEGNet model

In [ ]:
def build_eegnet_model(num_classes, channels, timesteps, dropout_rate=0.5):
    input_layer = Input(shape=(channels, timesteps, 1))

    # temporal convolution
    block1 = Conv2D(4, (1, 32), padding='same', use_bias=False, kernel_regularizer=regularizers.l2(1e-3))(input_layer)
    block1 = BatchNormalization()(block1)

    #Depthwise spatial convolution
    block1 = DepthwiseConv2D((channels, 1), use_bias=False, depth_multiplier=1, depthwise_constraint=tf.keras.constraints.max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = Dropout(dropout_rate)(block1)

    # classification head
    flatten_layer = Flatten()(block1)
    dense_layer = Dense(num_classes,
        kernel_constraint=tf.keras.constraints.max_norm(0.25))(flatten_layer)
    output_layer = Activation('softmax')(dense_layer)

    return Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Reshape data
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test_eegnet = X_test_norm[..., np.newaxis]
print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=NUM_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

# Define the earlystopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

Shape of data for EEGNet: (32, 248, 4453, 1)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 248, 4453, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 248, 4453, 4)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 248, 4453, 4)   │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 1, 4453, 4)     │           992 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1, 4453, 4)     │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1, 4453, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 1113, 4)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 1113, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4452)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │        17,812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,964 (74.08 KB)

 Trainable params: 18,948 (74.02 KB)

 Non-trainable params: 16 (64.00 B)

In [ ]:
history_eegnet = eegnet_model.fit(
    X_train_eegnet,
    y_train,
    epochs=20,  # We can still set a high number, but early stopping will likely stop it earlier
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

print("\nEvaluating EEGNet model on test sets")

loss_eegnet, acc_eegnet = eegnet_model.evaluate(X_test_eegnet, y_test, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 1: {acc_eegnet * 100:.2f}%")

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 29s 10s/step - accuracy: 0.3650 - loss: 1.3218 - val_accuracy: 0.5714 - val_loss: 1.0642
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 40s 9s/step - accuracy: 0.8042 - loss: 0.6018 - val_accuracy: 0.5714 - val_loss: 0.8309
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 32s 10s/step - accuracy: 0.8933 - loss: 0.4097 - val_accuracy: 0.5714 - val_loss: 0.7647
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 37s 11s/step - accuracy: 0.8783 - loss: 0.3033 - val_accuracy: 0.5714 - val_loss: 0.7600
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 28s 11s/step - accuracy: 0.8517 - loss: 0.2867 - val_accuracy: 0.7143 - val_loss: 0.7272
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 40s 11s/step - accuracy: 0.8308 - loss: 0.3797 - val_accuracy: 0.8571 - val_loss: 0.6750
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 39s 10s/step - accuracy: 1.0000 - loss: 0.1135 - val_accuracy: 1.0000 - val_loss: 0.6399
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 41s 10s/step - accuracy: 0.9525 - loss: 0.1128 - val_accuracy: 1.0000 - val_loss: 0.6156
E

Now I wanna try to use the same models for the cross subject and see if, even if very simple, they still work well with a satisfying accuracy.

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Deep Learning/Final Project data/Cross'

TRAIN_DIR = os.path.join(BASE_PATH, 'train/')
TEST1_DIR = os.path.join(BASE_PATH, 'test1/')
TEST2_DIR = os.path.join(BASE_PATH, 'test2/')
TEST3_DIR = os.path.join(BASE_PATH, 'test3/')

# Each file has 248 sensor readings (rows) and 35624 time steps (columns)
N_CHANNELS = 248
N_TIMESTEPS = 35624

# The 4 states we want to classify
TASKS = ['rest', 'task_motor', 'task_story_math', 'task_working_memory']
# Map tasks to integer labels
task_to_label = {task: i for i, task in enumerate(TASKS)}

In [ ]:
# Function to load data from a list of file paths
def load_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        # Extractin the label
        filename = file_path.split('/')[-1]

        #handling the different task naming conventions
        if 'rest' in filename:
            labels.append(task_to_label['rest'])
        elif 'motor' in filename:
            labels.append(task_to_label['task_motor'])
        elif 'story' in filename or 'math' in filename:
             labels.append(task_to_label['task_story_math'])
        elif 'working' in filename or 'memory' in filename:
            labels.append(task_to_label['task_working_memory'])
        else:
            # iff a file doesn't match
            print(f"Could not determine task for file: {filename}")
            continue

        with h5py.File(file_path, 'r') as f:
            # Instead of guessing the dataset name, we get the first key from the file
            # This is robust because we know there is only one dataset per file[cite: 10].
            dataset_name = list(f.keys())[0]
            matrix = f[dataset_name][()]
            data.append(matrix)

    #convert to numpy arrays
    return np.array(data), np.array(labels)

In [ ]:
train_files = glob.glob(f"{TRAIN_DIR}/*.h5")
test1_files = glob.glob(f"{TEST1_DIR}/*.h5")
test2_files = glob.glob(f"{TEST2_DIR}/*.h5")
test3_files = glob.glob(f"{TEST3_DIR}/*.h5")

X_train, y_train = load_data(train_files)
X_test1, y_test1 = load_data(test1_files)
X_test2, y_test2 = load_data(test2_files)
X_test3, y_test3 = load_data(test3_files)


print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test1: {X_test1.shape}")
print(f"Uniqe labels: {np.unique(y_train)}")
print(f"Number of training samples: {len(X_train)}")

Shape of X_train: (64, 248, 35624)
Shape of y_train: (64,)
Shape of X_test1: (16, 248, 35624)
Uniqe labels: [0 1 2 3]
Number of training samples: 64


In [ ]:
# --- Downsampling ---
# The original sample rate is 2034Hz
#take every 10th sample
DOWNSAMPLE_FACTOR = 10
X_train_ds = X_train[:, :, ::DOWNSAMPLE_FACTOR]
X_test1_ds = X_test1[:, :, ::DOWNSAMPLE_FACTOR]
X_test2_ds = X_test2[:, :, ::DOWNSAMPLE_FACTOR]
X_test3_ds = X_test3[:, :, ::DOWNSAMPLE_FACTOR]

N_TIMESTEPS_DS = X_train_ds.shape[2]
print(f"Original number of time steps: {N_TIMESTEPS}")
print(f"Downsampled number of time steps: {N_TIMESTEPS_DS}")

print("\nNormalizing data...")
X_train_norm = normalize_data(X_train_ds)
X_test1_norm = normalize_data(X_test1_ds)
X_test2_norm = normalize_data(X_test2_ds)
X_test3_norm = normalize_data(X_test3_ds)

#DL models in Keras often expect the channel dimension last
#reshaping from (samples, channels, timesteps) to (samples, timesteps, channels)
X_train_final = np.transpose(X_train_norm, (0, 2, 1))
X_test1_final = np.transpose(X_test1_norm, (0, 2, 1))
X_test2_final = np.transpose(X_test2_norm, (0, 2, 1))
X_test3_final = np.transpose(X_test3_norm, (0, 2, 1))

print("Normalization complete.")
print(f"Final shape of training data for the model: {X_train_final.shape}")

Original number of time steps: 35624
Downsampled number of time steps: 3563

Normalizing data...
Normalization complete.
Final shape of training data for the model: (64, 3563, 248)


CNN

In [ ]:
INPUT_SHAPE = (N_TIMESTEPS_DS, N_CHANNELS)
NUM_CLASSES = len(TASKS)

cnn_model = build_cnn_model(INPUT_SHAPE, NUM_CLASSES)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Use sparse CE because our labels are integers
    metrics=['accuracy']
)
cnn_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 3563, 16)       │        11,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3563, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            68 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,052 (47.08 KB)

 Trainable params: 12,020 (46.95 KB)

 Non-trainable params: 32 (128.00 B)

In [ ]:
history_cnn = cnn_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,      # smaller batch size for better generalization
    validation_split=0.2 # 20% of training data for validation
)

# Evaluate on test Sets
loss1_cnn, acc1_cnn = cnn_model.evaluate(X_test1_final, y_test1, verbose=0)
print(f"accuracy on test set 1: {acc1_cnn * 100:.2f}%")
loss2_cnn, acc2_cnn = cnn_model.evaluate(X_test2_final, y_test2, verbose=0)
print(f"accuracy on test set 2: {acc2_cnn * 100:.2f}%")
loss3_cnn, acc3_cnn = cnn_model.evaluate(X_test3_final, y_test3, verbose=0)
print(f"accuracy on test set 3: {acc3_cnn * 100:.2f}%")

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - accuracy: 0.1315 - loss: 3.2965 - val_accuracy: 0.3077 - val_loss: 1.4855
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2259 - loss: 2.4379 - val_accuracy: 0.3846 - val_loss: 1.4315
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2239 - loss: 2.1661 - val_accuracy: 0.4615 - val_loss: 1.3212
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5409 - loss: 1.5369 - val_accuracy: 0.6923 - val_loss: 1.1985
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5879 - loss: 1.1488 - val_accuracy: 0.6923 - val_loss: 1.0804
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.5263 - loss: 1.2536 - val_accuracy: 0.9231 - val_loss: 0.9860
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.6979 - loss: 1.2915 - val_accuracy: 0.8462 - val_loss: 0.9269
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6578 - loss: 1.2161 - val_accuracy: 0.9231 - val_loss: 0.8843

LSTM

In [ ]:
cnn_lstm_model = build_cnn_lstm_model(INPUT_SHAPE, NUM_CLASSES)

cnn_lstm_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_lstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 3563, 32)       │        39,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3563, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1781, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,348 (192.77 KB)

 Trainable params: 49,284 (192.52 KB)

 Non-trainable params: 64 (256.00 B)

In [ ]:
print("Starting CNN-LSTM model training...")
history_cnn_lstm = cnn_lstm_model.fit(
    X_train_final,
    y_train,
    epochs=20,
    batch_size=16,
    validation_split=0.2
)

# Evaluate on test sets
loss1_hybrid, acc1_hybrid = cnn_lstm_model.evaluate(X_test1_final, y_test1, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 1: {acc1_hybrid * 100:.2f}%")
loss2_hybrid, acc2_hybrid = cnn_lstm_model.evaluate(X_test2_final, y_test2, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 2: {acc2_hybrid * 100:.2f}%")
loss3_hybrid, acc3_hybrid = cnn_lstm_model.evaluate(X_test3_final, y_test3, verbose=0)
print(f"Hybrid Model Accuracy on Test Set 3: {acc3_hybrid * 100:.2f}%")

Starting CNN-LSTM model training...
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 931ms/step - accuracy: 0.4831 - loss: 1.3928 - val_accuracy: 0.3077 - val_loss: 1.3517
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 826ms/step - accuracy: 0.7413 - loss: 1.0688 - val_accuracy: 0.6154 - val_loss: 1.1791
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 826ms/step - accuracy: 0.8331 - loss: 0.8433 - val_accuracy: 0.6154 - val_loss: 1.0966
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 817ms/step - accuracy: 0.7987 - loss: 0.8500 - val_accuracy: 0.6154 - val_loss: 1.0337
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 845ms/step - accuracy: 0.9332 - loss: 0.6535 - val_accuracy: 0.6154 - val_loss: 0.9628
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 829ms/step - accuracy: 0.8712 - loss: 0.6498 - val_accuracy: 0.6154 - val_loss: 0.9124
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 849ms/step - accuracy: 0.9186 - loss: 0.5564 - val_accuracy: 0.6923 - val_loss: 0.8626
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 848ms/step - accuracy: 0.8618 - loss: 0.626

EEGNet

In [ ]:
# Reshape data
X_train_eegnet = X_train_norm[..., np.newaxis]
X_test1_eegnet = X_test1_norm[..., np.newaxis]
X_test2_eegnet = X_test2_norm[..., np.newaxis]
X_test3_eegnet = X_test3_norm[..., np.newaxis]

print(f"Shape of data for EEGNet: {X_train_eegnet.shape}")

eegnet_model = build_eegnet_model(
    num_classes=NUM_CLASSES,
    channels=N_CHANNELS,
    timesteps=N_TIMESTEPS_DS
)

eegnet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

eegnet_model.summary()

# Define the earlystopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

Shape of data for EEGNet: (64, 248, 3563, 1)


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 248, 3563, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 248, 3563, 4)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 248, 3563, 4)   │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 3563, 4)     │           992 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 1, 3563, 4)     │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1, 3563, 4)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 1, 890, 4)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1, 890, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │        14,244 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,396 (60.14 KB)

 Trainable params: 15,380 (60.08 KB)

 Non-trainable params: 16 (64.00 B)

In [ ]:
print("\nStarting EEGNet model training with Early Stopping...")
history_eegnet = eegnet_model.fit(
    X_train_eegnet,
    y_train,
    epochs=50,  # We can still set a high number, but early stopping will likely stop it earlier
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stopping]
)

print("\nEvaluating EEGNet model on test sets")
loss1_eegnet, acc1_eegnet = eegnet_model.evaluate(X_test1_eegnet, y_test1, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 1: {acc1_eegnet * 100:.2f}%")

loss2_eegnet, acc2_eegnet = eegnet_model.evaluate(X_test2_eegnet, y_test2, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 2: {acc2_eegnet * 100:.2f}%")

loss3_eegnet, acc3_eegnet = eegnet_model.evaluate(X_test3_eegnet, y_test3, verbose=0)
print(f"EEGNet Model Accuracy on Test Set 3: {acc3_eegnet * 100:.2f}%")


Starting EEGNet model training with Early Stopping...
Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.3903 - loss: 1.3417 - val_accuracy: 0.5385 - val_loss: 0.9388
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.7866 - loss: 0.5540 - val_accuracy: 0.5385 - val_loss: 0.8757
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9071 - loss: 0.3145 - val_accuracy: 0.6154 - val_loss: 0.8739
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9050 - loss: 0.3319 - val_accuracy: 0.6154 - val_loss: 0.7957
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.8962 - loss: 0.2731 - val_accuracy: 0.6154 - val_loss: 0.7416
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9228 - loss: 0.2414 - val_accuracy: 0.6154 - val_loss: 0.6931
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9614 - loss: 0.1882 - val_accuracy: 0.6154 - val_loss: 0.6881
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.9619 - loss: 0.